# Homework 2
Elzette Janse van Rensburg

## Mc-Call Job Search Model with Separation

We consider here a very simple job-search model, with separation. Our goal here consists in solving this model using a policy iteration algorithm.

### Model

There is a single worker who can be either employed ("e") or unemployed ("u") in any period.

When unemployed, the jobless worker receives unemployment benefits $c_t=\alpha>0$ in every period as long as he stays unemployed. He also receives a salary offer $w_t$ which is drawn from a discrete i.i.d. distribution, which takes values $w_1, ..., w_K$ with probabilities $p_1, ... p_K$ respectively.

When an unemployed worker accepts an offer in period $t$, he gets the salary $w_t$ and becomes employed. He then keeps his salary $w_t$ as long as he stays employed (for $s\geq t$, $c_s=w_t$ if $t$ is the date at which worker got the current job); in each period he has a probability $\lambda$ of becoming unemployed in the next period and remains employed otherwise.

When a worker receives a given amount $x$ his perceived utility is $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ with $\gamma>1.0$. A worker discounts the future at a rate $\beta \in [0,1[$. As a result, in any period $t_0$ workers seek to maximize $\sum_{t\geq t_0}^{\infty} U(c_t)$.



__Define a parameter type `Parameter`, with fields $\alpha$, $\beta$, $\gamma$, $K$, $\sigma$, $\lambda$. Create a parameter variable $\omega$ with $\alpha=0.5$, $\beta=0.96$, $\gamma=4$, $K=10$, $\sigma=0.6$, $\lambda=0.015$__




In [256]:
struct Parameter
    α::Float64
    β::Float64
    γ::Int64    
    K::Int
    σ::Float64
    λ::Float64 
end

ω = Parameter(0.5,0.96,4,10,0.6,0.015)

ω.K

10

__What is the uniform distribution, whose mean is 1 and standard deviation is $\sigma$? Write a function `discrete_uniform(σ::Float64, K::Int64)::Tuple{Vector{Float64, Float64}}` to discretize it, using $K$ points. The function should return two vectors `w` and `p` of floats of the same size `K`. Check the results satisfy the conditions (uniformity, standard deviation).__



 A uniform distribution on the interval (a, b) has a mean $μ = (a+b)/2 $ and a variance $ σ^2= (b-a)^2/12$. Thereby, a uniform distribution with standard deviation $\sigma$ and  mean $μ = 1$ is a uniform distribution on the interval $(1-\frac{\sqrt{12} \sigma}{2}, 1+\frac{\sqrt{12} \sigma}{2}$). Below is a discete version of the distribution. 

In [257]:
function discrete_uniform(σ::Float64, K::Int64)
    
    a = 1 - (12)^0.5*σ/2
    b = 1 + (12)^0.5*σ/2
    w = [LinRange(a, b, K)...]
    p = [1/K for i=1:K]
    
    return (w,p) #w: certain wage  p:probablilty of getting wage 
end 


discrete_uniform (generic function with 1 method)

In [258]:
using LinearAlgebra

In [259]:
w, p = discrete_uniform(ω.σ, ω.K)

μ = dot(w,p)  #mean
σ = ( dot( (w-ones( ω.K )).^2 , p) )^0.5 #standard deviation 


println("The mean of the discrete distribution is:",μ)
println("The standard deviation of the discrete distribution is:",σ)

The mean of the discrete distribution is:0.9999999999999998
The standard deviation of the discrete distribution is:0.6633249580710798


The optimal decision of a worker is characterized by two value functions: $V^E(w)$ the value of being employed at wage $w$ and $V^U(w)$ the value of being unemployed, while receiving job offer $w_t$. In Julia, both $V^U$ and $V^E$ will be represented by arrays `V_U` and `V_E` of size `K`.

A policy `g` is a binary choice in the unemployed state: accept or reject an offer $w_t$. It will then naturally be represented by a boolean array (type `zeros(Bool, K)` to initialize one).

__Given a policy $g$, write down the recursive equation which defines the corresponding value functions $V^{U,g}(w)$ and $V^{E,g}(w)$.__



The value of the next update will be given by: 
$$
V(w) = \begin{cases}
V_E(w)  \text{ if } g=1 \\
V_U(w)  \text{ if } g=0
\end{cases}
$$


Then, 


$$
V^{E,g}_{t}(w) = util(w) +\beta \left[ (1-\lambda)*V^E_{t+1}(w)+\lambda * V^U_{t+1}(w) \right] 
$$

and 

$$
V^{U,g}_{t}(w) = util( \alpha ) +\beta \left( \sum_{w_i \in W} max\left(V^U_{t+1}(w_i),V^E_{t+1}(w_i) \right) p(w_i) \right) 
$$

Note that the sum is an expectation over the future wages. From here on, I'm not quite going to use the guidance given by notebook, but take an alternative approach which I think is equivalent. Particularly, I will always be updating and returning both $V^E$ and $V^U$ not $V$ itself. I found it more convenient to work with. 

__Write a function which takes a guess $V^{U,g}(w)$ and $V^{E,g}(w)$ and a policy function $g$ as arguments (and other model parameters) and updates the values, according to the updating equations. This function could have signature `value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})::Vector{Float64}` where the returned vector has the same size as the supplied ones.__





In [294]:
function utilt(w,ω)
    return ( w.^(1-ω.γ) )./(1-ω.γ)
end


function next_value( g,p,w,ω,V_u,V_e )

            
    #calculating next step 
    U_next =  ( utilt(ω.α,ω) + ω.β*dot( max.(V_u,V_e) , p ) )*ones(ω.K) 
    E_next =   utilt(w,ω) + ω.β*( (1-ω.λ)*V_e + ω.λ*V_u )

    return (U_next,E_next)
end


next_value (generic function with 1 method)

In [295]:
k = utilt(1,ω)

-0.3333333333333333

__Write a function `eval_policy(g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Tuple{Vector{Float64}, Vector{Float64}}` which iterates on `value_update` find the values that satisfy the evaluation equations for policy `g`.__



In [296]:
# taken notebook 7
distance(A::Vector{Float64}, B::Vector{Float64}) = maximum( (u)->abs(u[1]-u[2]), zip(A[1],B[1]))

distance (generic function with 1 method)

In [297]:
function eval_policy(g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64,V_U,V_E)

    V_fin  = [ g[i] ? V_U[i] : V_E[i] for i in 1:ω.K ] 
    V = ones(ω.K)
    
    
    V_us = [V_U] #initialize 
    V_es = [ V_E] #initialize 
    
    val_V = [V_fin ] #keep track of values unemployed 
    T_max =1000000
    i = 0
    while (distance(V,V_fin) > η) & (i<T_max)

        V_U,V_E =  next_value(g,p,w,ω,V_us[end],V_es[end])
        
        push!(  V_us,V_U)
        push!( V_es,V_E)
        
        V_fin = [ g[i] ? V_E[i] : V_U[i] for i in 1:ω.K ]
        push!(val_V, V_fin)
        i += 1
  
        end 
    
    return V_us[end], V_es[end]
end 


eval_policy (generic function with 2 methods)

__Write a function `improve_policy(g::Vector{Bool}, V_U::Vector{Bool}, V_E::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Vector{Float64}` which returns the improved policy given guesses for the value function(s) at for $t+1$.__



In [319]:
#this is still no bueno NEED TO FIX 


function improve_policy(g::Vector{Bool}, V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)        
    
    V_Enew, V_Unew = eval_policy(g, ω, w, p, η,V_U,V_E)  
    return convert(Vector{Bool},V_Enew .> V_Unew)
end 


improve_policy (generic function with 3 methods)

In [320]:
l = [1,2,3,4]

j = [6,1,2,3]

k = [l .> j]

k = [ k[i] == 1 for i in 1:length(k)]


1-element Array{Bool,1}:
 0

__Implement the policy function algorithm. Print the successive approximation errors and comment on the convergence speed.__



In [336]:
function policy_algorithm(ω)
    #initialize value functions 
    #initialize policy 
    val_u = ones(ω.K)*0.3
    val_e = ones(ω.K)*0.5
    g = ones(ω.K) 
    g = [1==g[i] for i in 1:ω.K]
    gs = [g]
    η = 0.000001
    vals = [ ones(ω.K), [g[i] ? val_e[i] : val_u[i] for i in 1:ω.K ] ]
    
    Tmax = 10000000
    i = 1

    
    while (distance( vals[end],vals[end-1] ) > η) & (i < Tmax)
        
        #get max val using end policy 
        val_u, val_e = eval_policy(gs[end], ω, w, p, η, val_u,val_e)

        

        #update end policy to match max val
        g_new = improve_policy( gs[end], val_u, val_e, ω, w, p, η )
        push!(gs,g_new)
        
        val_fin = [ g_new[i] ? val_e[i] : val_u[i] for i in 1:ω.K ]
        push!(vals, val_fin)
        i += 1
        
    end    
    
    dif = [distance(vals[i],vals[i-1]) for i in 2:length(vals)]   
    return  vals, dif 
end 
    

policy_algorithm (generic function with 2 methods)

In [337]:
vals, dif = policy_algorithm(ω)

([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [88074.91803380546, 22444.291674581007, 23232.78963561277, 23292.002456908256, 23305.09483613894, 23309.534141468812, 23311.438463685983, 23312.387558754664, 23312.91261555881, 23313.22631967954], [88074.91803380546, 22444.291674581007, 23232.78963561277, 23292.002456908256, 23305.09483613894, 23309.534141468812, 23311.438463685983, 23312.387558754664, 23312.91261555881, 23313.22631967954]], [0.5, 88074.41803380546, 0.0])

This is converging extremely fast, I am not sure why or if it is right. 


<!-- 
## Bonus: reinforcement learning version

The previous solution relied on perfect knowledge of the model: the probabilities, the effect of the policies... In this (optional section) we consider another way to solve the problem, where the agent only uses past experience to update the value and to improve its job-search policy.
 -->

## Neoclassical growth model (2)

We consider here, another deterministic version of the neoclassical growth model, but propose a slightly different solution method.

A representative agent uses capital $k_t$ to produce $y_t$ using the following production function:

$$y_t = k_t^{\alpha}$$

He chooses to consume an amount $c_t \in ]0, y_t]$ and invests what remains:

$$i_t = y_t - c_t$$.

He accumulates capital $k_t$ according to:

$$k_{t+1} = \left( 1-\delta \right) k_{t} + i_{t}$$

where $\delta$ is the depreciation rate and $i_t$ is the amount invested.

The goal of the representative agent is to maximize:

$$\sum_{t\geq 0} \beta^t U(c_t)$$

where $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ and $\beta<1$ is the discount factor.

Since the problem is time homogenous, the value function only depends only on available capital and satisfies the following Bellman equation:

$$V\left(\underbrace{k}_{k_t}\right) = \max_{c\in[0,1[} U(c) + \beta V\left(\underbrace{(1-\delta)k + \underbrace{(k^{\alpha}-c)}_{y_{t+1}}}_{k_{t+1}}\right)$$

Our goal is to obtain a smooth approximation of $k$ and $V$ by using interpolations techniques.

For this model, using the dynamic first-order conditions, ne can show the deterministic steady-state of the model satisfies $1=\beta \left( (1-\delta) + \alpha k^{\alpha -1} \right)$. 

__Create a suitable Parameter type to hold the paramters. Write a function `steady_state(p::Parameter)` to compute the steady-state capital `kbar` and the corresponding steady-state consumption `cbar`__ 



In [338]:
struct Param
    α::Float64
    β::Float64
    γ::Float64
    δ::Float64
end

Using the fact that the deterministic steady-state of the model satisfies $1=\beta \left( (1-\delta) + \alpha k^{\alpha -1} \right)$, we obtain $kbar = \left( (\frac{1}{β} + δ - 1)\frac{1}{α} \right)^{\frac{1}{(α-1)}} $ and by using the fact that in the equilibrium $k_{t+1}=k_{t}$ combined with the otehr equations that govern the investment and production rules we obtain $cbar = kbar^α - δ*kbar$.

In [340]:
function steady_state(p::Param)
    kbar = ((1/p.β + p.δ - 1)/p.α)^(1/(p.α-1)) 
    cbar = kbar^p.α - p.δ*kbar 
    return kbar, cbar
end

steady_state (generic function with 1 method)

__Set $N=10$ and define a reasonable grid `kgrid=range(kmin, kmax; length=N)` to approximate capital $k$.__



In [342]:
N = 10
p = Param(0.5, 0.95, 3.0, 0.04) 
kbar, cbar = steady_state(p)


kmin = kbar/2;
kmax = 1.5* kbar;

kgrid=range(kmin, kmax; length = N);

__The unknown value function is represented as a `N` elements arrays. Define `Vi(k,p)=δ * k^α` and compute the initial guess `V0 = [Vi(k) for k in kgrid]`. Define a finer grid `ktest=range(kmin, kmax;length=1000)` and find the values of `Vi` on it by  using `Interpolations.jl` library to interpolate `V0` between the points of `kgrid`.__



In [345]:
using Pkg
Pkg.add("Interpolations")


  Resolving package versions...
   Updating `~/.juliapro/JuliaPro_v1.4.0-1/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.juliapro/JuliaPro_v1.4.0-1/environments/v1.4/Manifest.toml`
 [no changes]


In [346]:
using Interpolations

┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1260


In [348]:
function Vi(k,p) 
    return (p.δ * k^p.α)^(1-p.γ)/((1-p.β)*(1-p.γ))
end

Vi (generic function with 1 method)

In [354]:
V0 = [Vi(k,p) for k in kgrid] #get correspong Vi for all value in grid

ktest= range(kmin, kmax; length = 1000)
V0test = [Vi(k,p) for k in ktest] 

#interpolation 

interp = interpolate( V0, BSpline(Linear()) ) #estimate using linear spline 
interp_scaled= scale(interp, kgrid)

10-element scale(interpolate(::Array{Float64,1}, BSpline(Linear())), (14.567729855371908:3.2372733011937576:43.703189566115725,)) with element type Float64:
 -429.0304709141268
 -351.024930747922
 -297.0210952482417
 -257.41828254847604
 -227.13377871924362
 -203.22495990669168
 -183.87020182034004
 -167.88148861857138
 -154.45096952908568
 -143.0101569713756

__Compute a Bellman improvement function `bellman(V0::Vector{Float64}, p::Parameter, kgrid)::Tuple{Vector{Float64}, Vector{Float64}}` which does the following steps:__

- take an initial guess `V0` for the value function

- at each grid point from kvec, optimize nonlinearly, the function $c \rightarrow U(c) + \beta V\left((1-\delta)k + (k^{\alpha}-c)\right)$ for each capital level in the grid `kvec`. In this expression the function `V()` interpolates `V0` defined on `kvec` on any point `k` so that the resulting function is continuous. 

- return the updated value and investment rules



In [355]:
#Note that all code in this exercise is a slight modification form code in previous notebooks


function bellman(V0::Vector{Float64}, p::Param, kgrid)
  
    V = deepcopy(V0)
    C = zeros(length(V0))
    for (n,k) in enumerate(kgrid)
        nn0 = -1
        gv = -Inf
        copt = - 1.0
        for (nn,kk) in enumerate(kgrid)
            y = k^p.α
            i = kk - (1 - p.δ) * k 
            c = y - i
            
            if c < 0 # can't have negative consumption ! 
                continue
            end
            

            
            interp = interpolate(V0, BSpline(Linear()))
            interpscaled = scale(interp, kgrid)
            esitp = extrapolate(interpscaled,Line());
            
            var = (1-p.δ) * kk + kk^p.α - c
            v = c^(1-p.γ)/(1-p.γ) + p.β * esitp(var)
            

            if v > gv
                gv = v
                copt = c
                nn0 = nn
            end
        end
        V[n] = gv
        C[n] = copt
    end
    return V, C
end

bellman (generic function with 1 method)

__Write a value function interation function `vfi(N, p)` which solves the model defined by parameter `p` using the value function algorithm. The function should return the value function and the policy rule.__



In [356]:
function vfi(V0, p, tol, kgrid)
    V0 = deepcopy(V0)
    V,C = bellman(V0, p, kgrid)
    
    while distance(V,V0) > tol
        V0 = V
        V, C = bellman(V, p, kgrid)
    end
    return V, C
end

vfi (generic function with 1 method)

__Plot the solution__

__Bonus 1: plot a graph showing the convergence back to the steady-state__

__Bonus 2: implement the policy iteration algorithm by adding an evaluation step in the `vfi` function.__


In [357]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
ERROR: LoadError: LoadError: EOFError: read end of file
Stacktrace:
 [1] read(::IOStream, ::Type{Int64}) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
 [2] parse_cache_header(::IOStream) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
 [3] stale_cachefile(::String, ::String) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
 [4] _require_search_from_serialized(::Base.PkgId, ::String) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
 [5] _require(::Base.PkgId) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
 [6] require(::Base.PkgId) at /Applications/JuliaPro-1.4.0-1.app/Contents/Resources/juli

ErrorException: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to /Users/Elzette/.juliapro/JuliaPro_v1.4.0-1/compiled/v1.4/Plots/ld3vC_F2mWv.ji.

In [358]:
tol = 0.00001
V, C = vfi(V0 ,p, tol, kgrid)


plot(kgrid, V)


plot(kgrid, C)






UndefVarError: UndefVarError: plot not defined

In [ ]:
tol = 0.001
V, C = vfi(V0test, p, tol, ktest)

plot(ktest, V)

plot(ktest, C)